 # 1. primera metodologia

### 1.0 crear datos sinteticos

In [67]:
import pandas as pd
import numpy as np

#paso 0: parametros
n_activos = 5
periodos=200 # este es T
n_portafolios = 10

# Paso 1: Crear el DataFrame de retornos (5 activos)
fechas = pd.date_range('1974-01-02', periods=periodos, freq='B')  # 20 días hábiles
np.random.seed(42)
data = np.random.randn(periodos, n_activos)  # Generación de retornos aleatorios
df = pd.DataFrame(data, index=fechas, columns=[f'Activo_{i+1}' for i in range(n_activos)])
df


,Activo_1,Activo_2,Activo_3,Activo_4,Activo_5
1974-01-02,0.496714,-0.138264,0.647689,1.523030,-0.234153
1974-01-03,-0.234137,1.579213,0.767435,-0.469474,0.542560
1974-01-04,-0.463418,-0.465730,0.241962,-1.913280,-1.724918
1974-01-07,-0.562288,-1.012831,0.314247,-0.908024,-1.412304
1974-01-08,1.465649,-0.225776,0.067528,-1.424748,-0.544383
...,...,...,...,...,...
1974-10-02,-1.889541,-0.452306,-2.423879,-1.583903,0.760415
1974-10-03,0.785800,0.425458,-0.966976,-0.047711,-0.003603
1974-10-04,-1.158365,1.503398,0.877362,-0.220964,0.026886
1974-10-07,0.208383,-2.041735,-0.247177,-0.681984,-1.001620


In [68]:
# Paso 2: Crear portafolios con pesos aleatorios para 48 portafolios (48 combinaciones)
  # Número de portafolios
pesos = np.random.rand(n_portafolios, n_activos)  # 48 portafolios, 5 activos
pesos = pesos / pesos.sum(axis=1)[:, np.newaxis]  # Normalizamos para que sumen 1

# Multiplicamos los retornos por los pesos para obtener los portafolios
portafolios = np.dot(df.values, pesos.T)

# Paso 3: Crear un DataFrame para los portafolios con los valores calculados
df_portafolios = pd.DataFrame(portafolios, index=fechas, columns=[f'Portafolio_{i+1}' for i in range(n_portafolios)])
df_portafolios


,Portafolio_1,Portafolio_2,Portafolio_3,Portafolio_4,Portafolio_5,Portafolio_6,Portafolio_7,Portafolio_8,Portafolio_9,Portafolio_10
1974-01-02,0.941081,0.541698,0.613026,0.428033,0.150113,0.588335,0.386217,0.645351,0.030778,0.499727
1974-01-03,0.182081,0.134410,0.371005,0.422966,0.728846,0.480728,0.662202,0.238773,0.974221,0.527886
1974-01-04,-0.837204,-1.024621,-0.843339,-0.729371,-0.671868,-1.121651,-0.821116,-0.892498,-0.721543,-0.805194
1974-01-07,-0.416748,-0.734159,-0.712635,-0.631950,-0.859158,-0.883899,-0.719781,-0.595746,-0.943501,-0.677673
1974-01-08,-0.460909,-0.028106,-0.114908,0.073981,0.237604,-0.593746,-0.327496,-0.188580,-0.032407,-0.235160
...,...,...,...,...,...,...,...,...,...,...
1974-10-02,-1.822827,-1.146593,-1.368768,-1.235708,-0.868881,-0.981789,-0.992786,-1.380385,-0.574311,-1.203331
1974-10-03,-0.287352,0.063482,0.194468,0.036333,0.389905,0.092275,-0.068416,-0.098046,0.301542,-0.002226
1974-10-04,0.222481,-0.213760,0.181103,0.125506,0.336979,0.433653,0.543592,0.052281,0.689355,0.388930
1974-10-07,-0.515840,-0.450478,-0.802835,-0.621851,-1.027795,-1.093845,-0.981060,-0.526467,-1.325574,-0.864410


## 1.1 Ortogonalización

In [69]:

# Paso 4: Calcular la matriz de covarianzas con una ventana móvil
ventana = 15  # Tamaño de la ventana móvil (ajustado para que funcione con los 20 días)
cov_matrices = []
for i in range(ventana, len(df_portafolios)):
    ventana_datos = df_portafolios.iloc[i-ventana:i]
    cov_matrices.append(ventana_datos.cov().values)



In [74]:
import numpy as np
import pandas as pd

# Lista para almacenar los portafolios dinámicos
portafolios_dinamicos = []

for t, cov in enumerate(cov_matrices):
    cov_matrix = cov

    # Paso 5: Descomponer la matriz de covarianzas (autovalores y autovectores)
    eigenvalues, eigenvectors = np.linalg.eigh(cov_matrix)

    # Paso 6: Ordenar los autovalores y autovectores en orden descendente
    idx = eigenvalues.argsort()[::-1]  # Índices ordenados
    eigenvalues = eigenvalues[idx]
    eigenvectors = eigenvectors[:, idx]

    # Paso 7: Normalizar los eigenvectores seleccionados
    eigenvectors_normalized = eigenvectors / np.linalg.norm(eigenvectors, axis=0)

    # Paso 8: Crear portafolios ortogonales a partir de los eigenvectores normalizados
    df_ortogonales = pd.DataFrame(
        np.dot(df_portafolios.values, eigenvectors_normalized),
        index=fechas,
        columns=[f'portafolio_ortogonal_{i+1}' for i in range(eigenvectors_normalized.shape[1])]
    )
    
    # Paso 9: Determinar dinámicamente el número de eigenportafolios principales (l)
    mediana_eigenvalores = np.median(eigenvalues)  # Calcular la mediana de los eigenvalores
    l = sum(eigenvalues > mediana_eigenvalores)  # Contar cuántos eigenvalores superan la mediana

    # Paso 10: Seleccionar los primeros l portafolios ortogonales
    df_ortogonales_l = df_ortogonales.iloc[:, :l]
    portafolios_dinamicos.append(df_ortogonales_l)

    # Mostrar los pesos de los portafolios principales
    #print(f"Número de eigenportafolios seleccionados dinámicamente l={l} en t={t}")
print(f"Pesos de los portafolios principales (autovectores normalizados):")
print(eigenvectors_normalized[:, :l])  # Los pesos de los primeros 'l' portafolios ortogonales

# Mostrar los primeros l portafolios ortogonales
print(df_ortogonales_l.head())



Pesos de los portafolios principales (autovectores normalizados):
[[-0.35999845  0.54336709  0.24635749 -0.04995517 -0.41354045]
 [-0.25635665  0.21956245 -0.43613725  0.10204573  0.66287235]
 [-0.34810363  0.03409415 -0.04165754 -0.66883739 -0.09372811]
 [-0.29751218  0.08119247 -0.39841786  0.19100736 -0.15142732]
 [-0.31348223 -0.41414867 -0.37968231 -0.2747186  -0.16090021]
 [-0.32743141 -0.10077504  0.59355174 -0.21046757  0.53505592]
 [-0.31053806 -0.09415205  0.23774493  0.54382608 -0.07055272]
 [-0.29965756  0.29979695 -0.14809194  0.18071924  0.09856183]
 [-0.31041199 -0.60688119  0.05901242  0.20894736 -0.07751898]
 [-0.32699567  0.00822054  0.10110135  0.10691925 -0.15809592]]
            portafolio_ortogonal_1  portafolio_ortogonal_2  \
1974-01-02               -1.544376                0.707025   
1974-01-03               -1.493092               -0.752518   
1974-01-04                2.662212               -0.135549   
1974-01-07                2.249575                0.437

In [106]:
print(eigenvectors_normalized[:, :])

[[-0.35999845  0.54336709  0.24635749 -0.04995517 -0.41354045  0.00475635
  -0.35536552  0.03452119  0.39694995 -0.2363934 ]
 [-0.25635665  0.21956245 -0.43613725  0.10204573  0.66287235  0.01948804
  -0.34952304  0.24018306  0.20154533  0.15871437]
 [-0.34810363  0.03409415 -0.04165754 -0.66883739 -0.09372811 -0.28086803
  -0.14903428 -0.01252638 -0.42865     0.36715377]
 [-0.29751218  0.08119247 -0.39841786  0.19100736 -0.15142732 -0.03030252
   0.1365837   0.24206936 -0.51485168 -0.58608896]
 [-0.31348223 -0.41414867 -0.37968231 -0.2747186  -0.16090021  0.55114554
   0.00858313 -0.29863702  0.28862444 -0.09148063]
 [-0.32743141 -0.10077504  0.59355174 -0.21046757  0.53505592  0.1549036
   0.16774252 -0.02748478 -0.07010975 -0.37675521]
 [-0.31053806 -0.09415205  0.23774493  0.54382608 -0.07055272  0.29695974
  -0.34304781 -0.25544252 -0.39860722  0.32779394]
 [-0.29965756  0.29979695 -0.14809194  0.18071924  0.09856183 -0.30953911
   0.51108654 -0.61141886  0.11481097  0.10946933]
 

In [28]:
# siempre es l=5, en esencia no cambia

## ahora que :( o sea tengo 79 matrices en portafolios dinamicos, se supone que cada uno es una realizacion de la verdadera distribucion??

In [82]:
len(portafolios_dinamicos)

185

#### crear matriz de retornos

In [84]:

import pandas as pd

# Crear una lista vacía para almacenar las filas
filas = []

# Supongamos que portafolios_dinamicos es una lista de DataFrames
for i in range(len(portafolios_dinamicos)):
    # Tomamos la fila correspondiente, incrementando el índice con el valor de 'i'
    fila = portafolios_dinamicos[i].iloc[ventana-1 + i]  # Ajustamos el índice de la fila
    filas.append(fila)

# Convertir la lista de filas en un DataFrame
resultado_df = pd.DataFrame(filas)

# Mostrar el DataFrame resultante
resultado_df



,portafolio_ortogonal_1,portafolio_ortogonal_2,portafolio_ortogonal_3,portafolio_ortogonal_4,portafolio_ortogonal_5
1974-01-22,-1.322194,-0.077769,0.943435,-0.229652,0.230963
1974-01-23,0.484650,0.098091,0.340345,0.394651,0.047661
1974-01-24,-0.279806,0.141966,0.152480,-0.080906,-0.333476
1974-01-25,-0.462468,-0.575499,0.049071,-0.157572,0.120432
1974-01-28,0.095375,-0.718762,0.183409,0.198089,0.031774
...,...,...,...,...,...
1974-10-01,0.637535,0.395139,-0.210806,-0.071946,0.157062
1974-10-02,3.781473,-0.299094,0.051596,0.167831,-0.076425
1974-10-03,-0.127820,-0.551475,-0.095944,-0.204609,0.141325
1974-10-04,-0.840399,-0.594352,0.466445,0.231845,-0.198009


### matriz de retornos binarios

In [118]:
## matriz de 1's y 0's
#t=1,2,3,..185
resultado_df_binario= resultado_df.apply(lambda row: row == row.max(), axis=1).astype(int)
resultado_df_binario

,portafolio_ortogonal_1,portafolio_ortogonal_2,portafolio_ortogonal_3,portafolio_ortogonal_4,portafolio_ortogonal_5
1974-01-22,0,0,1,0,0
1974-01-23,1,0,0,0,0
1974-01-24,0,0,1,0,0
1974-01-25,0,0,0,0,1
1974-01-28,0,0,0,1,0
...,...,...,...,...,...
1974-10-01,1,0,0,0,0
1974-10-02,1,0,0,0,0
1974-10-03,0,0,0,0,1
1974-10-04,0,0,1,0,0


## 1.2 ADTS

### 1.2.1 ADTS sin ortogonalizar

In [153]:
# sin ortogonalizar
import numpy as np
from scipy.stats import beta
import pandas as pd

def adaptive_discounted_thompson_sampling(returns, gamma, w):
    """
    Adaptive Discounted Thompson Sampling (Algorithm 1).

    Parameters:
        returns (pd.DataFrame): DataFrame donde las filas son días y las columnas son portafolios,
                                y los valores son los retornos diarios.
        gamma (float): Factor de descuento (0 < gamma <= 1).
        w (int): Tamaño de la ventana deslizante (1 <= w <= T).

    Returns:
        actions (list): Índices de los portafolios seleccionados cada día.
        observed_rewards (list): Retornos observados para los portafolios seleccionados.
    """
    T, K = returns.shape  # T: número de días, K: número de portafolios
    returns_binario= returns.apply(lambda row: row == row.max(), axis=1).astype(int)
    
    # Inicializar parámetros alpha y beta
    alpha = np.ones(K)  # α_k inicializado en 1
    beta_params = np.ones(K)  # β_k inicializado en 1

    actions = []  # Almacenar los portafolios elegidos
    observed_rewards = []  # Almacenar los retornos observados
    eventos=[]

    for t in range(T):
        # Step 2: Calcular θ_k(t) y θ̌_k(t) para cada portafolio
        theta = []
        theta_tilde = []
        for k in range(K):
            # θ_k(t) ~ Beta(α_k + 1, β_k + 1)
            theta_k = beta.rvs(alpha[k] + 1, beta_params[k] + 1)
            theta.append(theta_k)
            
            # θ̌_k(t) ~ Beta(α_k^w, β_k^w), utilizando las últimas w recompensas
            if t < w:
                ventana_reward_t=returns_binario.iloc[0:t-1, k]  # Retorna lo mas que pueda
            else:
                ventana_reward_t=returns_binario.iloc[t-w-1:t-1, k]  # Si t es mayor que w, retorna los datos t-w
        
            alpha_w = np.count_nonzero (ventana_reward_t.values)  # Suponiendo recompensas positivas son éxitos
            beta_w = len(ventana_reward_t.values)- alpha_w
            theta_tilde_k = beta.rvs(alpha_w + 1, beta_w + 1)
            theta_tilde.append(theta_tilde_k)
            
        
        # Step 6: Elegir el portafolio I(t) = argmax_k f(θ_k(t), θ̌_k(t))
        f_values = [(theta[k] + theta_tilde[k])/2 for k in range(K)]
        I_t = np.argmax(f_values)  # Índice del portafolio seleccionado con la ponderacion
        actions.append(I_t)

        # Step 7: Observar el retorno r_t del portafolio seleccionado
        reward_t = returns.iloc[t, I_t]
        observed_rewards.append(reward_t)

        
        # Step 8: Calcular X_t (1 si retorno es positivo, 0 en caso contrario)
        X_t = 1 if reward_t == returns.iloc[t, :].max() else 0
        eventos.append(X_t)

        # Step 9: Actualizar Beta(α_k, β_k) para todos los portafolios
        for k in range(K):
            if k == I_t:
                alpha[k] = gamma * alpha[k] + X_t
                beta_params[k] = gamma * beta_params[k] + (1 - X_t)
            else:
                alpha[k] = gamma*alpha[k]
                beta_params[k] = gamma*beta_params[k]
        
    return actions, observed_rewards, eventos


# Simular DataFrame con tus retornos (cargar tus datos reales aquí)
returns = df_portafolios

# Parámetros del algoritmo
gamma = 0.9  # Factor de descuento
w = 10  # Tamaño de la ventana deslizante

# Ejecutar el algoritmo
actions, observed_rewards, eventos = adaptive_discounted_thompson_sampling(returns, gamma, w)

# Resultados
print("Portafolios seleccionados cada día:", actions)
#print("Retornos observados:", observed_rewards)
print("eventos:", eventos)
numero_de_unos = eventos.count(1)
print(" numero aciertos:", numero_de_unos)

Portafolios seleccionados cada día: [8, 4, 8, 5, 0, 4, 0, 2, 7, 5, 6, 4, 5, 0, 1, 5, 0, 0, 0, 2, 1, 3, 6, 8, 7, 6, 1, 9, 5, 6, 0, 5, 2, 8, 9, 3, 1, 0, 4, 8, 4, 1, 7, 5, 4, 2, 7, 6, 1, 8, 2, 0, 9, 8, 7, 3, 8, 4, 0, 8, 0, 9, 6, 0, 3, 8, 1, 5, 2, 6, 8, 8, 1, 6, 1, 2, 0, 1, 5, 4, 2, 5, 7, 5, 1, 4, 3, 5, 0, 3, 1, 8, 7, 9, 5, 4, 8, 8, 4, 6, 8, 0, 5, 0, 2, 3, 3, 6, 5, 0, 8, 7, 4, 2, 1, 0, 0, 0, 0, 2, 8, 7, 1, 9, 4, 2, 6, 1, 0, 1, 1, 3, 5, 4, 0, 1, 8, 8, 0, 5, 5, 5, 0, 6, 4, 8, 4, 1, 5, 7, 3, 8, 0, 1, 0, 0, 0, 9, 2, 0, 4, 6, 1, 0, 8, 3, 1, 4, 5, 7, 3, 6, 6, 9, 2, 7, 5, 8, 1, 1, 4, 6, 0, 1, 6, 0, 0, 1, 2, 1, 0, 1, 1, 1, 4, 0, 2, 9, 7, 1]
eventos: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,

### 1.2.2 ADTS solo actualizando el arma que se dispara con ortogonalizacion

In [154]:
import numpy as np
from scipy.stats import beta
import pandas as pd

def adaptive_discounted_thompson_sampling(returns, gamma, w):
    """
    Adaptive Discounted Thompson Sampling (Algorithm 1).

    Parameters:
        returns (pd.DataFrame): DataFrame donde las filas son días y las columnas son portafolios,
                                y los valores son los retornos diarios.
        gamma (float): Factor de descuento (0 < gamma <= 1).
        w (int): Tamaño de la ventana deslizante (1 <= w <= T).

    Returns:
        actions (list): Índices de los portafolios seleccionados cada día.
        observed_rewards (list): Retornos observados para los portafolios seleccionados.
    """
    T, K = returns.shape  # T: número de días, K: número de portafolios
    returns_binario=  returns.apply(lambda row: row == row.max(), axis=1).astype(int)
    # Inicializar parámetros alpha y beta
    alpha = np.ones(K)  # α_k inicializado en 1
    beta_params = np.ones(K)  # β_k inicializado en 1

    actions = []  # Almacenar los portafolios elegidos
    observed_rewards = []  # Almacenar los retornos observados
    eventos=[]

    for t in range(T):
        # Step 2: Calcular θ_k(t) y θ̌_k(t) para cada portafolio
        theta = []
        theta_tilde = []
        for k in range(K):
            # θ_k(t) ~ Beta(α_k + 1, β_k + 1)
            theta_k = beta.rvs(alpha[k] + 1, beta_params[k] + 1)
            theta.append(theta_k)
            
            # θ̌_k(t) ~ Beta(α_k^w, β_k^w), utilizando las últimas w recompensas
            if t < w:
                ventana_reward_t=returns_binario.iloc[0:t-1, k]  # Retorna lo mas que pueda
            else:
                ventana_reward_t=returns_binario.iloc[t-w-1:t-1, k]  # Si t es mayor que w, retorna los datos t-w
        
            alpha_w = np.count_nonzero (ventana_reward_t.values)  # Suponiendo recompensas positivas son éxitos
            beta_w = len(ventana_reward_t.values)- alpha_w
            theta_tilde_k = beta.rvs(alpha_w + 1, beta_w + 1)
            theta_tilde.append(theta_tilde_k)
            
        
        # Step 6: Elegir el portafolio I(t) = argmax_k f(θ_k(t), θ̌_k(t))
        f_values = [(theta[k] + theta_tilde[k])/2 for k in range(K)]
        I_t = np.argmax(f_values)  # Índice del portafolio seleccionado con la ponderacion
        actions.append(I_t)

        # Step 7: Observar el retorno r_t del portafolio seleccionado
        reward_t = returns.iloc[t, I_t]
        observed_rewards.append(reward_t)

        

        # Step 8: Calcular X_t (1 si retorno es positivo, 0 en caso contrario)
        X_t = 1 if reward_t == returns.iloc[t, :].max() else 0
        eventos.append(X_t)

        # Step 9: Actualizar Beta(α_k, β_k) para todos los portafolios
        for k in range(K):
            if k == I_t:
                alpha[k] = gamma * alpha[k] + X_t
                beta_params[k] = gamma * beta_params[k] + (1 - X_t)
            else:
                alpha[k] = gamma*alpha[k]
                beta_params[k] = gamma*beta_params[k]
        
    return actions, observed_rewards, eventos


# Simular DataFrame con tus retornos (cargar tus datos reales aquí)
returns = resultado_df

# Parámetros del algoritmo
gamma = 0.9  # Factor de descuento
w = 10  # Tamaño de la ventana deslizante

# Ejecutar el algoritmo
actions, observed_rewards, eventos = adaptive_discounted_thompson_sampling(returns, gamma, w)

# Resultados
print("Portafolios seleccionados cada día:", actions)
#print("Retornos observados:", observed_rewards)
print("eventos:", eventos)
numero_de_unos = eventos.count(1)
print(" numero aciertos:", numero_de_unos)

Portafolios seleccionados cada día: [0, 2, 2, 4, 0, 2, 2, 3, 4, 2, 4, 3, 0, 4, 4, 1, 0, 0, 3, 3, 3, 0, 0, 3, 2, 0, 0, 1, 1, 4, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 2, 0, 4, 3, 2, 3, 2, 2, 3, 2, 2, 2, 2, 4, 2, 0, 3, 0, 0, 0, 0, 4, 2, 0, 1, 3, 1, 0, 3, 1, 1, 3, 4, 0, 0, 2, 4, 0, 2, 0, 0, 3, 0, 1, 0, 4, 1, 0, 0, 3, 4, 0, 0, 0, 0, 0, 1, 4, 2, 2, 4, 3, 1, 0, 1, 0, 0, 1, 1, 0, 0, 4, 0, 0, 0, 1, 2, 3, 0, 0, 0, 1, 0, 0, 0, 0, 4, 1, 2, 0, 3, 1, 1, 2, 0, 4, 0, 4, 1, 0, 3, 1, 0, 2, 1, 2, 1, 1, 2, 0, 4, 1, 2, 3, 1, 1, 4, 4, 0, 3, 1, 1, 1, 1, 1, 1, 1, 1, 3, 1, 1, 1, 1, 4, 0, 4, 1, 4, 0, 1, 2]
eventos: [0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,

### 1.2.3 ADTS actualizando todas las armas(disparadas y no disparadas)

In [155]:
#pesos fijos y lo que elijo es un portafolio dado
import numpy as np
from scipy.stats import beta
import pandas as pd

def adaptive_discounted_thompson_sampling(returns, gamma, w):
    """
    Adaptive Discounted Thompson Sampling (Algorithm 1).

    Parameters:
        returns (pd.DataFrame): DataFrame donde las filas son días y las columnas son portafolios,
                                y los valores son los retornos diarios.
        gamma (float): Factor de descuento (0 < gamma <= 1).
        w (int): Tamaño de la ventana deslizante (1 <= w <= T).

    Returns:
        actions (list): Índices de los portafolios seleccionados cada día.
        observed_rewards (list): Retornos observados para los portafolios seleccionados.
    """
    T, K = returns.shape  # T: número de días, K: número de portafolios
    returns_binario=  returns.apply(lambda row: row == row.max(), axis=1).astype(int)
    # Inicializar parámetros alpha y beta
    alpha = np.ones(K)  # α_k inicializado en 1
    beta_params = np.ones(K)  # β_k inicializado en 1

    actions = []  # Almacenar los portafolios elegidos
    observed_rewards = []  # Almacenar los retornos observados
    eventos=[]

    for t in range(T):
        # Step 2: Calcular θ_k(t) y θ̌_k(t) para cada portafolio
        theta = []
        theta_tilde = []
        for k in range(K):
            # θ_k(t) ~ Beta(α_k + 1, β_k + 1)
            theta_k = beta.rvs(alpha[k] + 1, beta_params[k] + 1)
            theta.append(theta_k)
            
            # θ̌_k(t) ~ Beta(α_k^w, β_k^w), utilizando las últimas w recompensas
            if t < w:
                ventana_reward_t=returns_binario.iloc[0:t-1, k]  # Retorna lo mas que pueda
            else:
                ventana_reward_t=returns_binario.iloc[t-w-1:t-1, k]  # Si t es mayor que w, retorna los datos t-w
        
            alpha_w = np.count_nonzero (ventana_reward_t.values)  # Suponiendo recompensas positivas son éxitos
            beta_w = len(ventana_reward_t.values)- alpha_w
            theta_tilde_k = beta.rvs(alpha_w + 1, beta_w + 1)
            theta_tilde.append(theta_tilde_k)
            
        
        # Step 6: Elegir el portafolio I(t) = argmax_k f(θ_k(t), θ̌_k(t))
        f_values = [(theta[k] + theta_tilde[k])/2 for k in range(K)]
        I_t = np.argmax(f_values)  # Índice del portafolio seleccionado con la ponderacion
        actions.append(I_t)

        # Step 7: Observar el retorno r_t del portafolio seleccionado
        reward_t = returns.iloc[t, I_t]
        observed_rewards.append(reward_t)

        # Step 8: Calcular X_t (1 si retorno es positivo, 0 en caso contrario)
        X_t = 1 if reward_t == returns.iloc[t, :].max() else 0
        eventos.append(X_t)
        

        # Step 9: Actualizar Beta(α_k, β_k) para todos los portafolios
        for k in range(K): #acá actualizó tanto el que disparo como el máximo
            if k == I_t:
                alpha[k] = gamma * alpha[k] + 1
                beta_params[k] = gamma * beta_params[k] 
            else:
                alpha[k] =  gamma * alpha[k] 
                beta_params[k] = gamma * beta_params[k] + 1
        
    return actions, observed_rewards, eventos


# Simular DataFrame con tus retornos (cargar tus datos reales aquí)
returns = resultado_df

# Parámetros del algoritmo
gamma = 0.9  # Factor de descuento
w = 10  # Tamaño de la ventana deslizante

# Ejecutar el algoritmo
actions, observed_rewards, eventos = adaptive_discounted_thompson_sampling(returns, gamma, w)

# Resultados
print("Portafolios seleccionados cada día:", actions)
#print("Retornos observados:", observed_rewards)
print("eventos:", eventos)
numero_de_unos = eventos.count(1)
print(" numero aciertos:", numero_de_unos)

Portafolios seleccionados cada día: [1, 1, 3, 0, 2, 1, 2, 0, 2, 0, 0, 2, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
eventos: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1,

In [ ]:
### de la metodologia 1 me falta 1. evaluar la convergencia, 2. pensar como meto todas las realizaciones

In [ ]:
### markowitz : sacar la matriz de varianza y covarianza de los portafolios y la media esperada